## Обработка UDPipe

In [3]:
!C:\Users\Eiko\PycharmProjects\NLP_19_20\udpipe-1.2.0-bin\bin-win64\udpipe.exe --input horizontal --output conllu \
--tokenize --tag --parse \
C:\Users\Eiko\PycharmProjects\NLP_19_20\russian-syntagrus-ud-2.4-190531.udpipe \
< C:\Users\Eiko\PycharmProjects\NLP_19_20\hw2_collocations\testset2.txt > C:\Users\Eiko\PycharmProjects\NLP_19_20\hw2_collocations\testset2.conllu

Loading UDPipe model: done.


In [64]:
corpus = "testset2.conllu"

In [65]:
# Вытащим деревья

trees = []

with open(corpus, 'r', encoding='utf-8') as f:
    parsed_sents = f.read().split('\n\n')

    for sent in parsed_sents:
        tree = [line for line in sent.split('\n') if line and line[0] != '#']
        trees.append('\n'.join(tree))

In [66]:
token_n, lemma_n, pos_n, grammar_n, synt_n = 1, 2, 3, 5, 7

In [95]:
from collections import Counter

verb_lemmas = []
verb_tokens = []
for s in trees:
    s = s.split('\n')
    for token in s:
        if token:
            token = token.split('\t')
            if token[pos_n] == 'VERB':
                verb_lemmas.append(token[lemma_n])
                verb_tokens.append((token[lemma_n], token[token_n].lower()))
lemmas_counted = Counter(verb_lemmas)

# Список глагольных лемм, которые часто встречаются
verbs = {token[1]:lemmas_counted[token[0]] for token in verb_tokens if lemmas_counted[token[0]] >= 50}
        

In [96]:
verbs

{'заявил': 258,
 'подал': 274,
 'просит': 144,
 'признать': 345,
 'стал': 213,
 'сообщил': 217,
 'обратились': 139,
 'отказался': 117,
 'рассматривать': 99,
 'рассказал': 54,
 'рассмотрит': 90,
 'может': 349,
 'стать': 213,
 'начались': 74,
 'обвинили': 461,
 'решил': 86,
 'стало': 213,
 'пытались': 89,
 'используются': 57,
 'напомним': 97,
 'получили': 127,
 'обвинивший': 461,
 'передался': 68,
 'признал': 345,
 'направленные': 105,
 'обвинил': 461,
 'удалось': 61,
 'направило': 105,
 'сказал': 98,
 'имеет': 94,
 'просил': 144,
 'сделал': 77,
 'рассматривал': 99,
 'обжалуемое': 102,
 'сообщила': 217,
 'отказал': 59,
 'сделала': 77,
 'решили': 86,
 'говорится': 139,
 'обратился': 139,
 'сказала': 98,
 'посчитал': 55,
 'есть': 136,
 'оставила': 61,
 'дав': 70,
 'обвинив': 461,
 'признала': 345,
 'обвинила': 461,
 'удовлетворив': 173,
 'заявила': 258,
 'вынес': 153,
 'подала': 274,
 'потребовав': 173,
 'требующей': 173,
 'рассматриваться': 99,
 'имел': 94,
 'является': 113,
 'было': 136,

In [97]:
from nltk.parse import DependencyGraph

bigrams = []
trees_without_roots = []

for s in trees:
    try:
        g = DependencyGraph(s, top_relation_label='root')
        for t in g.triples():
            head = t[0][0]
            dep = t[2][0]
            dep_postag = t[2][1]
            rel = t[1]
        
            if head in verbs and dep_postag == "NOUN" and rel == "obj":
                bigrams.append((head, dep))
    except:
        trees_without_roots.append(s)

print("Корявых предложений: ", len(trees_without_roots))
print("Всего предложений: ", len(trees))
            

Корявых предложений:  18
Всего предложений:  6482


In [190]:
len(bigrams)

2059

## Извлечение коллокаций

Я думаю, что в коллокациях все-таки значима форма слова, поэтому не буду лемматизировать глаголы.

In [191]:
# Токенизация

import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import pymorphy2
morph = pymorphy2.MorphAnalyzer()


def preprocessing(input_text, normal_form=False, del_stopwords=False, del_digit=True):
    """
    :input: raw text
        1. lowercase, del punctuation, tokenize
        2. del stopwords
        3. del digits
    :return: tokens
    """
    if del_stopwords:
        russian_stopwords = set(stopwords.words('russian'))
    words = [x.lower().strip(string.punctuation+'»«–…') for x in word_tokenize(input_text)]
    words = [x for x in words if x]
    
    if normal_form:
        lemmas = [morph.parse(x)[0].normal_form for x in words if x]
        lemmas_arr = []
        for lemma in lemmas:
            if del_stopwords:
                if lemma in russian_stopwords:
                    continue
            if del_digit:
                if lemma.isdigit():
                    continue
            lemmas_arr.append(lemma)
        return lemmas_arr
    return words

In [192]:
with open('testset2.txt', 'r', encoding='utf-8') as f:
    corpus_text = f.read()
    
finder = collocations.BigramCollocationFinder.from_documents([preprocessing(corpus_text)])


In [193]:
finder.apply_ngram_filter(lambda w1, w2: (w1, w2) not in bigrams)

In [194]:
finder.nbest(collocations.BigramAssocMeasures().likelihood_ratio, 5)

[('удовлетворил', 'иск'),
 ('принял', 'решение'),
 ('удовлетворил', 'ходатайство'),
 ('вынес', 'решение'),
 ('обжаловать', 'решение')]

In [195]:
ll_scored = finder.score_ngrams(collocations.BigramAssocMeasures().likelihood_ratio)
dice_scored = finder.score_ngrams(collocations.BigramAssocMeasures().dice)
pmi_scored = finder.score_ngrams(collocations.BigramAssocMeasures().pmi)

## Золотой стандарт

In [231]:
ll_best = finder.nbest(collocations.BigramAssocMeasures().likelihood_ratio, 100) 
pmi_best = finder.nbest(collocations.BigramAssocMeasures().pmi, 100)
dice_best = finder.nbest(collocations.BigramAssocMeasures().dice, 100)

gold = set(ll_best) & set(pmi_best) & set(dice_best)

In [232]:
# Встречаются в топе всех трех метрик
gold

{('дав', 'согласие'),
 ('даст', 'киллеру'),
 ('делает', 'вывод'),
 ('заявив', 'отвод'),
 ('объявить', 'перерыв'),
 ('обязав', 'горсовет'),
 ('подадут', 'апелляцию'),
 ('получила', 'повестку'),
 ('принявший', 'ислам'),
 ('принять', 'резолюцию'),
 ('провела', 'проверку'),
 ('прошли', 'прения'),
 ('рассматривал', 'прошение'),
 ('рассматривала', 'просьбу'),
 ('удовлетворит', 'требование')}

In [233]:
gold_norm = {(morph.parse(x[0])[0].normal_form, morph.parse(x[1])[0].normal_form): x for x in gold if x}


In [234]:
verb_coll = []
with open("verb_coll.txt", 'r', encoding="utf-8") as f:
    text = f.read()

lines = text.split("\n")
for line in lines:
    entry = line.split("\t")
    if len(entry) > 2:
        coll = entry[2]
        words = coll.split()
        if len(words) == 2:
            coll_lemm = (morph.parse(words[0])[0].normal_form, morph.parse(words[1])[0].normal_form)
            verb_coll.append(coll_lemm)

In [235]:
# Пересечение со словарем глагольной сочетаемости
inter = set(gold_norm) & set(verb_coll)
inter

{('дать', 'согласие'),
 ('делать', 'вывод'),
 ('объявить', 'перерыв'),
 ('принять', 'ислам'),
 ('принять', 'резолюция'),
 ('провести', 'проверка'),
 ('удовлетворить', 'требование')}

In [236]:
keys = set(gold_norm) - inter
for key in keys:
    print(gold_norm[key])

('заявив', 'отвод')
('получила', 'повестку')
('рассматривал', 'прошение')
('подадут', 'апелляцию')
('обязав', 'горсовет')
('прошли', 'прения')
('рассматривала', 'просьбу')
('даст', 'киллеру')


In [237]:
golden_standard = set()
for key in inter:
    golden_standard.add(gold_norm[key])

Добавим коллокаций

In [238]:
additions = (('заявив', 'отвод'), # устойчивое словосочетание, характерное для юридического текста
('получила', 'повестку'), # устойчивое словосочетание, характерное для юридического текста
('рассматривал', 'прошение'), # устойчивое словосочетание, характерное для юридического текста
('подадут', 'апелляцию'), # устойчивое словосочетание, характерное для юридического текста
# ('обязав', 'горсовет') скорее всего, вылезло вверх из-за деепричастия и редкого слова
('прошли', 'прения'), # устойчивое словосочетание, характерное для юридического текста
('рассматривала', 'просьбу')) # устойчивое словосочетание, характерное для юридического текста
# ('даст', 'киллеру') это вообще не прямой объект

for elem in additions:
    golden_standard.add(elem)

In [239]:
golden_standard

{('дав', 'согласие'),
 ('делает', 'вывод'),
 ('заявив', 'отвод'),
 ('объявить', 'перерыв'),
 ('подадут', 'апелляцию'),
 ('получила', 'повестку'),
 ('принявший', 'ислам'),
 ('принять', 'резолюцию'),
 ('провела', 'проверку'),
 ('прошли', 'прения'),
 ('рассматривал', 'прошение'),
 ('рассматривала', 'просьбу'),
 ('удовлетворит', 'требование')}

## Подсчет ранговой корреляции

In [240]:
# Сделаем ранги

ll_ranked = {elem[0]: i for i, elem in enumerate(ll_scored[:100])}
pmi_ranked = {elem[0]: i for i, elem in enumerate(pmi_scored[:100])}
dice_ranked = {elem[0]: i for i, elem in enumerate(dice_scored[:100])}

# У нас нет каких-то оснований для ранжирования слов внутри золотого стандарта, поэтому я просто отсортирую его.
golden_standard = sorted(golden_standard)

In [251]:
gold_r = list(range(len(golden_standard)))

In [252]:
gold_ranked_pmi = [pmi_ranked[i] for i in golden_standard]
gold_ranked_ll = [ll_ranked[i] for i in golden_standard]
gold_ranked_dice = [dice_ranked[i] for i in golden_standard]

In [256]:
from random import shuffle
from scipy.stats import spearmanr

shuffle(gold_r)
print('PMI: ', spearmanr(gold_ranked_pmi, gold_r))
print('Loglikelihood: ', spearmanr(gold_ranked_ll, gold_r))
print('Dice: ', spearmanr(gold_ranked_dice, gold_r))

PMI:  SpearmanrResult(correlation=0.04395604395604395, pvalue=0.8866190227890784)
Loglikelihood:  SpearmanrResult(correlation=0.2967032967032967, pvalue=0.3249215884859752)
Dice:  SpearmanrResult(correlation=-0.3186813186813187, pvalue=0.2885831614447202)


Цифры выше не показывают примерно ничего, так как у нас не было нормального ранжирования внутри золотого стандарта. Давайте посмотрим, насколько ранги согласуются между собой вообще.

In [257]:
print('PMI и Dice: ', spearmanr(gold_ranked_pmi, gold_ranked_dice))
print('Loglikelihood и PMI: ', spearmanr(gold_ranked_ll, gold_ranked_pmi))
print('Dice и Loglikelihood: ', spearmanr(gold_ranked_dice, gold_ranked_ll))

PMI и Dice:  SpearmanrResult(correlation=0.5439560439560439, pvalue=0.05464653454086551)
Loglikelihood и PMI:  SpearmanrResult(correlation=-0.11538461538461539, pvalue=0.7073898735142602)
Dice и Loglikelihood:  SpearmanrResult(correlation=0.16483516483516483, pvalue=0.5904789886520052)


PMI и Dice коррелируют, а воот LL и Dice и LL и PMI не очень.